# Joins com DuckDB

* [FROM and JOIN Clauses](https://duckdb.org/docs/stable/sql/query_syntax/from)
* [NULL Values](https://duckdb.org/docs/stable/sql/data_types/nulls)

In [ ]:
import duckdb

In [ ]:
conn = duckdb.connect()

In [ ]:
conn.execute("""
    CREATE OR REPLACE TABLE pessoas (
        id INTEGER,
        nome TEXT,
        idade INTEGER,
        cidade_id INTEGER
    );
""")

conn.execute("""
    CREATE OR REPLACE TABLE cidades (
        id INTEGER,
        nome TEXT,
        estado TEXT
    );
""")

In [ ]:
conn.execute("""
    INSERT INTO pessoas VALUES
        (1, 'Ana', 22, 1),
        (2, 'Bruno', 35, 2),
        (3, 'Carla', 19, 3),
        (4, 'Daniel', 40, 2),
        (5, 'Eduarda', 28, NULL);
""")

conn.execute("""
    INSERT INTO cidades VALUES
        (1, 'Rio de Janeiro', 'RJ'),
        (2, 'São Paulo', 'SP'),
        (3, 'Belo Horizonte', 'MG'),
        (4, 'Curitiba', 'PR'),
        (5, 'Porto Alegre', 'RS');
""")

# INNER JOIN
retorna linhas onde a condição de junção é satisfeita em ambas as tabelas.

In [ ]:
display(conn.execute("""
    SELECT
        p.nome,
        p.idade,
        c.nome   AS cidade,
        c.estado
    FROM pessoas p
    INNER JOIN cidades c
        ON p.cidade_id = c.id;
""").df())

# LEFT JOIN
Retorna todas as linhas da tabela à esquerda e as correspondentes da tabela à direita. Se não houver correspondência, os valores da tabela da direita serão NULL.

In [ ]:
display(conn.execute("""
    SELECT
        p.nome,
        p.idade,
        c.nome   AS cidade,
        c.estado
    FROM pessoas p
    LEFT JOIN cidades c
        ON p.cidade_id = c.id;
""").df())

# RIGHT JOIN
Retorna todas as linhas da tabela à direita e as correspondentes da tabela à esquerda. Onde não houver correspondência, os valores da tabela da esquerda serão NULL.

In [ ]:
display(conn.execute("""
    SELECT
        p.nome,
        p.idade,
        c.nome AS cidade,
        c.estado
    FROM pessoas p
    RIGHT JOIN cidades c
        ON p.cidade_id = c.id;
""").df())

# FULL OUTER JOIN
Retorna todas as linhas de ambas as tabelas. Onde não houver correspondência, os campos ausentes são preenchidos com NULL.

In [ ]:
display(conn.execute("""
    SELECT
        p.nome,
        p.idade,
        c.nome   AS cidade,
        c.estado
    FROM pessoas p
    FULL OUTER JOIN cidades c
        ON p.cidade_id = c.id;
""").df())

# Produto cartesiano (CROSS JOIN ou JOIN sem ON)
O produto cartesiano retorna todas as combinações possíveis entre os registros das duas tabelas.

Se pessoas tem 5 linhas e cidades tem 5 linhas, o resultado terá 25 linhas (5 × 5).

In [ ]:
display(conn.execute("""
    SELECT
        p.nome AS pessoa,
        c.nome AS cidade
    FROM pessoas p, cidades c;
""").df())

In [ ]:
display(conn.execute("""
    SELECT
        p.nome AS pessoa,
        c.nome AS cidade
    FROM pessoas p
    CROSS JOIN cidades c;
""").df())

# Produto cartesiano parcial com chaves duplicadas na tabela da direita
Quando há duplicatas na chave da tabela da direita, o JOIN cria um produto cartesiano parcial entre os registros com o mesmo valor.

In [ ]:
# Adiciona duplicata na tabela cidades (cidade_id = 2)
conn.execute("""
    INSERT INTO cidades VALUES
        (2, 'São Paulo (Duplicata)', 'SP');
""")


In [ ]:
display(conn.execute("""
    SELECT
        p.nome AS pessoa,
        c.nome AS cidade
    FROM pessoas p
    INNER JOIN cidades c ON p.cidade_id = c.id
""").df())

In [ ]:
conn.close()